In [9]:
import featuretools as ft
import pandas as pad
import numpy as np
import os
lc_data=pad.read_csv("D:\Academic\Data Science\lc_data/clean_data.csv", low_memory=False)

interest_rate=lc_data['int_rate']


In [10]:
es=ft.EntitySet(id='interest_rate')

es.entity_from_dataframe(entity_id = 'credit_rating', dataframe = lc_data, index = 'id')

es.normalize_entity(base_entity_id='credit_rating', new_entity_id='borrower', index = 'member_id', 
additional_variables = ['loan_amnt','installment','emp_length','home_ownership','annual_inc','verification_status_joint','dti','dti_joint',
'open_acc','total_acc','annual_inc_joint'])



Entityset: interest_rate
  Entities:
    credit_rating [Rows: 887379, Columns: 18]
    borrower [Rows: 887379, Columns: 12]
  Relationships:
    credit_rating.member_id -> borrower.member_id

In [11]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'credit_rating', agg_primitives = ['min','max','mean','count','sum'], trans_primitives = ['cum_mean', 'cum_sum'],
                                       max_depth = 1, 
                                       verbose = 1, 
                                       n_jobs = 5)



Built 28 features
EntitySet scattered to workers in 5.474 seconds
Elapsed: 00:09 | Remaining: 00:00 | Progress: 100%|██████████████████████████████████████████| Calculated: 11/11 chunks


tornado.application - ERROR - Exception in Future <Future cancelled> after timeout
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\gen.py", line 970, in error_callback
    future.result()
concurrent.futures._base.CancelledError
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:57357 remote=tcp://127.0.0.1:57296>


In [12]:
feature_matrix.columns

Index(['member_id', 'funded_amnt', 'term', 'int_rate', 'grade', 'sub_grade',
       'loan_status', 'pymnt_plan', 'initial_list_status', 'out_prncp',
       'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'last_pymnt_amnt', 'application_type',
       'borrower.loan_amnt', 'borrower.installment', 'borrower.emp_length',
       'borrower.home_ownership', 'borrower.annual_inc',
       'borrower.verification_status_joint', 'borrower.dti',
       'borrower.dti_joint', 'borrower.open_acc', 'borrower.total_acc',
       'borrower.annual_inc_joint'],
      dtype='object')

In [13]:
feature_matrix.to_csv("D:\Academic\Data Science\lc_data/feature_matrix.csv")